In [1]:
import numpy as np
import pickle
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
with open('C:/Users/Z478SG/Desktop/Ecole/OpenClassrooms-Projet-7/modeling/data/06_models/lightgbm_model.pkl', 'rb') as file:
    model = pickle.load(file)

C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.3.0 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
data_path = "C:/Users/Z478SG/Desktop/Ecole/OpenClassrooms-Projet-7/modeling/data/03_primary/df_agg.csv"

In [4]:
data = pd.read_csv(data_path)

C:\Users\Z478SG\AppData\Local\Temp\ipykernel_11288\560202589.py:1: DtypeWarning: Columns (756,757,761,762,766,767,771,772,776,777,781,782,786,787,791,792) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path)


In [5]:
columns_to_remove = []

for col in data.columns:
    if data[col].dtype=='object':
        print(col)
        columns_to_remove.append(col)

CC_NAME_CONTRACT_STATUS_Active_MIN
CC_NAME_CONTRACT_STATUS_Active_MAX
CC_NAME_CONTRACT_STATUS_Approved_MIN
CC_NAME_CONTRACT_STATUS_Approved_MAX
CC_NAME_CONTRACT_STATUS_Completed_MIN
CC_NAME_CONTRACT_STATUS_Completed_MAX
CC_NAME_CONTRACT_STATUS_Demand_MIN
CC_NAME_CONTRACT_STATUS_Demand_MAX
CC_NAME_CONTRACT_STATUS_Refused_MIN
CC_NAME_CONTRACT_STATUS_Refused_MAX
CC_NAME_CONTRACT_STATUS_Sent proposal_MIN
CC_NAME_CONTRACT_STATUS_Sent proposal_MAX
CC_NAME_CONTRACT_STATUS_Signed_MIN
CC_NAME_CONTRACT_STATUS_Signed_MAX
CC_NAME_CONTRACT_STATUS_nan_MIN
CC_NAME_CONTRACT_STATUS_nan_MAX


In [6]:
data.drop(columns=columns_to_remove, inplace=True)

In [7]:
data.shape

(356251, 781)

In [8]:
for col in data.columns:
    if data[col].dtype=='object':
        print(col)

In [9]:
data.dtypes

SK_ID_CURR                              int64
TARGET                                float64
CODE_GENDER                             int64
FLAG_OWN_CAR                            int64
FLAG_OWN_REALTY                         int64
                                       ...   
CC_NAME_CONTRACT_STATUS_Signed_VAR    float64
CC_NAME_CONTRACT_STATUS_nan_MEAN      float64
CC_NAME_CONTRACT_STATUS_nan_SUM       float64
CC_NAME_CONTRACT_STATUS_nan_VAR       float64
CC_COUNT                              float64
Length: 781, dtype: object

In [10]:
data = data.astype('float16')

C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)
C:\Users\Z478SG\AppData\Roaming\Python\Python312\site-packages\pandas\core\dtypes\astype.py:133: RuntimeWarning: overflow encountered in cast
  return arr.astype(dtype, copy=True)


In [11]:
data.dtypes

SK_ID_CURR                            float16
TARGET                                float16
CODE_GENDER                           float16
FLAG_OWN_CAR                          float16
FLAG_OWN_REALTY                       float16
                                       ...   
CC_NAME_CONTRACT_STATUS_Signed_VAR    float16
CC_NAME_CONTRACT_STATUS_nan_MEAN      float16
CC_NAME_CONTRACT_STATUS_nan_SUM       float16
CC_NAME_CONTRACT_STATUS_nan_VAR       float16
CC_COUNT                              float16
Length: 781, dtype: object

In [12]:
train_data = data[data['TARGET'].notnull()]

In [13]:
train_data.shape

(307507, 781)

In [ ]:
# avoir train_data avec la même quantité de 0 ou de 1. Et regarder si proba tjrs pareil ou pas

# voir si y'a aussi cette erreur sur l'ancien notebook

# Faire tourner un autre modèle que LGBX

In [14]:
train_data = train_data[:10000]
train_data.shape

(10000, 781)

In [15]:
X = train_data.drop(['SK_ID_CURR', 'TARGET'], axis=1)
y = train_data['TARGET']

In [16]:
model.predict_proba(X)

array([[0.91466421, 0.08533579],
       [0.91988249, 0.08011751],
       [0.91986139, 0.08013861],
       ...,
       [0.91713096, 0.08286904],
       [0.91665911, 0.08334089],
       [0.91622034, 0.08377966]])

In [17]:
# Étape 1: Prédire les probabilités
proba_predictions = model.predict_proba(X)[:, 1]  # Prendre la probabilité de la classe 1


In [18]:
print(f"Min: {proba_predictions.min()}")
print(f"Mean: {proba_predictions.mean()}")
print(f"Max: {proba_predictions.max()}")

Min: 0.07976982829029503
Mean: 0.08081596845173342
Max: 0.08813962971194243


In [19]:
# Étape 2: Appliquer un seuil pour convertir les probabilités en prédictions binaires
def apply_threshold(proba_predictions, threshold):
    return (proba_predictions >= threshold).astype(int)

In [20]:
# Étape 3: Évaluer la précision du modèle
def evaluate_model(y_true, proba_predictions, threshold):
    binary_predictions = apply_threshold(proba_predictions, threshold)
    accuracy = accuracy_score(y_true, binary_predictions)
    return accuracy

In [21]:
# Exemple d'utilisation
threshold = 0.8  # Vous pouvez ajuster ce seuil manuellement
accuracy = evaluate_model(y, proba_predictions, threshold)
print(f"Précision du modèle avec un seuil de {threshold}: {accuracy}")

Précision du modèle avec un seuil de 0.8: 0.9225


In [22]:
# Pour tester différents seuils
thresholds = np.arange(0.79, 0.89, 0.01)
for threshold in thresholds:
    accuracy = evaluate_model(y, proba_predictions, threshold)
    print(f"Précision du modèle avec un seuil de {threshold}: {accuracy}")

Précision du modèle avec un seuil de 0.79: 0.9225
Précision du modèle avec un seuil de 0.8: 0.9225
Précision du modèle avec un seuil de 0.81: 0.9225
Précision du modèle avec un seuil de 0.8200000000000001: 0.9225
Précision du modèle avec un seuil de 0.8300000000000001: 0.9225
Précision du modèle avec un seuil de 0.8400000000000001: 0.9225
Précision du modèle avec un seuil de 0.8500000000000001: 0.9225
Précision du modèle avec un seuil de 0.8600000000000001: 0.9225
Précision du modèle avec un seuil de 0.8700000000000001: 0.9225
Précision du modèle avec un seuil de 0.8800000000000001: 0.9225
